In [1]:
import pandas as pd


weather = pd.read_csv("../Data/Weather_Data.csv")

In [2]:
df=weather 

In [3]:
import pandas as pd

# Supposons que votre DataFrame s'appelle df et que la colonne date/heure est 'Date/Time'
df['Date/Time'] = pd.to_datetime(df['Date/Time'])

# Extraire la date de la colonne 'Date/Time'
df['Date'] = df['Date/Time'].dt.date

# Calculer la moyenne de la température pour chaque jour
daily_avg_temp = df.groupby('Date')['Temp_C'].mean().reset_index()
daily_avg_temp['Temp_C'] = daily_avg_temp['Temp_C'].round(1)

# Afficher les moyennes de température quotidiennes
print(daily_avg_temp)


           Date  Temp_C
0    2012-01-01     0.6
1    2012-01-02     0.0
2    2012-01-03   -14.4
3    2012-01-04   -13.6
4    2012-01-05    -6.8
..          ...     ...
361  2012-12-27    -5.8
362  2012-12-28    -6.8
363  2012-12-29   -11.3
364  2012-12-30   -11.4
365  2012-12-31    -5.6

[366 rows x 2 columns]


In [4]:
# Calculer la moyenne de l'humidité et de la vitesse du vent par jour
daily_avg_humidity = df.groupby('Date')['Rel Hum_%'].mean().reset_index()
daily_avg_wind_speed = df.groupby('Date')['Wind Speed_km/h'].mean().reset_index()

# Renommer les colonnes
daily_avg_temp=daily_avg_temp.rename(columns={'Temp_C':'Temp_C'})
daily_avg_humidity = daily_avg_humidity.rename(columns={'Rel Hum_%': 'Rel Hum_%'})
daily_avg_wind_speed = daily_avg_wind_speed.rename(columns={'Wind Speed_km/h': 'Wind Speed_km/h'})

# Arrondir les valeurs moyennes et les convertir en entiers
daily_avg_humidity['Rel Hum_%'] = daily_avg_humidity['Rel Hum_%'].round().astype(int)
daily_avg_wind_speed['Wind Speed_km/h'] = daily_avg_wind_speed['Wind Speed_km/h'].round().astype(int)

# Fusionner les DataFrames
daily_avg_temp = pd.merge(daily_avg_temp, daily_avg_humidity, on='Date')
daily_avg_temp = pd.merge(daily_avg_temp, daily_avg_wind_speed, on='Date')

# Afficher le DataFrame final
print(daily_avg_temp)


           Date  Temp_C  Rel Hum_%  Wind Speed_km/h
0    2012-01-01     0.6         86               13
1    2012-01-02     0.0         63               31
2    2012-01-03   -14.4         55               22
3    2012-01-04   -13.6         63               10
4    2012-01-05    -6.8         70               12
..          ...     ...        ...              ...
361  2012-12-27    -5.8         89               35
362  2012-12-28    -6.8         83               18
363  2012-12-29   -11.3         89               15
364  2012-12-30   -11.4         80               22
365  2012-12-31    -5.6         85               13

[366 rows x 4 columns]


In [5]:
print(daily_avg_temp)


           Date  Temp_C  Rel Hum_%  Wind Speed_km/h
0    2012-01-01     0.6         86               13
1    2012-01-02     0.0         63               31
2    2012-01-03   -14.4         55               22
3    2012-01-04   -13.6         63               10
4    2012-01-05    -6.8         70               12
..          ...     ...        ...              ...
361  2012-12-27    -5.8         89               35
362  2012-12-28    -6.8         83               18
363  2012-12-29   -11.3         89               15
364  2012-12-30   -11.4         80               22
365  2012-12-31    -5.6         85               13

[366 rows x 4 columns]


In [6]:
# Vérifier à nouveau les valeurs nulles dans le DataFrame
null_values = daily_avg_temp.isnull().sum()

# Afficher le nombre de valeurs nulles pour chaque colonne
print(null_values)

Date               0
Temp_C             0
Rel Hum_%          0
Wind Speed_km/h    0
dtype: int64


In [7]:
daily_avg_temp["target_tmp"] = daily_avg_temp.shift(-1)["Temp_C"]
daily_avg_temp = daily_avg_temp.ffill()

daily_avg_temp["target_hum"] = daily_avg_temp.shift(-1)["Rel Hum_%"]
daily_avg_temp = daily_avg_temp.ffill()

daily_avg_temp["target_vent"] = daily_avg_temp.shift(-1)["Wind Speed_km/h"]
daily_avg_temp = daily_avg_temp.ffill()



In [8]:
daily_avg_temp

,Date,Temp_C,Rel Hum_%,Wind Speed_km/h,target_tmp,target_hum,target_vent
0,2012-01-01,0.6,86,13,0.0,63.0,31.0
1,2012-01-02,0.0,63,31,-14.4,55.0,22.0
2,2012-01-03,-14.4,55,22,-13.6,63.0,10.0
3,2012-01-04,-13.6,63,10,-6.8,70.0,12.0
4,2012-01-05,-6.8,70,12,-11.1,73.0,17.0
...,...,...,...,...,...,...,...
361,2012-12-27,-5.8,89,35,-6.8,83.0,18.0
362,2012-12-28,-6.8,83,18,-11.3,89.0,15.0
363,2012-12-29,-11.3,89,15,-11.4,80.0,22.0
364,2012-12-30,-11.4,80,22,-5.6,85.0,13.0


### LINEAR REGRESSION

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# Charger les données
X_train = daily_avg_temp.drop(['Date','target_vent','target_hum','target_tmp'], axis=1).iloc[:300]  # Premières 8771 lignes pour l'entraînement
y_train = daily_avg_temp['target_tmp'].iloc[:300]

X_test = daily_avg_temp.drop(['Date','target_vent','target_hum','target_tmp'], axis=1).iloc[-66:]  # Dernières 12 lignes pour le test
y_test = daily_avg_temp['target_tmp'].iloc[-66:]
# Choisissez le modèle de régression (dans cet exemple, une régression linéaire)
model = LinearRegression()

# Entraînez le modèle sur les données d'entraînement
model.fit(X_train, y_train)

predictions = model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)  # RMSE
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print('Mean Squared Error (MSE):', mse)
print('Root Mean Squared Error (RMSE):', rmse)
print('Mean Absolute Error (MAE):', mae)
print('R² Score:', r2)


Mean Squared Error (MSE): 13.830851551337044
Root Mean Squared Error (RMSE): 3.7189852851735035
Mean Absolute Error (MAE): 2.775234730776038
R² Score: 0.6171729706703061


In [10]:
import pandas as pd

# Faites des prédictions sur les données de test
predictions = model.predict(X_test)

# Arrondir les prédictions et les valeurs réelles à une seule décimale
predictions_rounded = [round(pred, 1) for pred in predictions]
y_test_rounded = [round(real, 1) for real in y_test]

# Afficher les prédictions et les valeurs réelles dans un DataFrame
predictions_df = pd.DataFrame({'Prédictions': predictions_rounded, 'Température Réelle': y_test_rounded})
print(predictions_df)


    Prédictions  Température Réelle
0          13.5                11.3
1          10.4                12.8
2           7.1                16.5
3          12.3                11.6
4          11.7                 7.4
..          ...                 ...
61        -10.4                -6.8
62         -6.7               -11.3
63        -10.3               -11.4
64        -11.9                -5.6
65         -4.4                -5.6

[66 rows x 2 columns]


## prediction humidity

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

# Divisez les données en ensembles d'entraînement et de test
X1_train = daily_avg_temp.drop(['Date','target_hum','target_tmp','target_vent'], axis=1).iloc[:300]  # Premières 7991 lignes pour l'entraînement
y1_train = daily_avg_temp['target_hum'].iloc[:300]

X1_test = daily_avg_temp.drop(['Date','target_hum','target_tmp','target_vent'], axis=1).iloc[-66:]  # Dernières 768 lignes pour le test
y1_test = daily_avg_temp['target_hum'].iloc[-66:]

# Choisissez le modèle de régression (dans cet exemple, RandomForestRegressor)
model1 =LinearRegression()

# Entraînez le modèle sur les données d'entraînement
model1.fit(X1_train, y1_train)

# Faites des prédictions sur les données de test
predictionss = model1.predict(X1_test)

# Évaluez les performances du modèle
mse1 = mean_squared_error(y1_test, predictionss)
rmse1 = mse1 ** 0.5  # Root Mean Squared Error
mae1 = mean_absolute_error(y1_test, predictionss)
r21 = r2_score(y1_test, predictionss)

print('Mean Squared Error:', mse1)
print('Root Mean Squared Error:', rmse1)
print('Mean Absolute Error:', mae1)
print('R² Score:', r21)

# Afficher les prédictions et les valeurs réelles dans un DataFrame
predictions_df1 = pd.DataFrame({'Prédictions': [round(pred, 1) for pred in predictionss], 
                               'Humidité Réelle': [round(real, 1) for real in y1_test]})
print(predictions_df1)


Mean Squared Error: 89.12965252202082
Root Mean Squared Error: 9.440850201227684
Mean Absolute Error: 7.357162322155458
R² Score: 0.053495030654125975
    Prédictions  Humidité Réelle
0          75.4             76.0
1          70.3             76.0
2          64.7             75.0
3          66.0             90.0
4          79.4             85.0
..          ...              ...
61         73.0             83.0
62         75.0             89.0
63         79.4             80.0
64         72.6             85.0
65         77.5             85.0

[66 rows x 2 columns]


## PREDICTION VENT


In [12]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Charger les données
X2_train = daily_avg_temp.drop(['Date','target_vent','target_tmp','target_hum'], axis=1).iloc[:300]
y2_train = daily_avg_temp['target_vent'].iloc[:300]

X2_test = daily_avg_temp.drop(['Date','target_vent','target_tmp','target_hum'], axis=1).iloc[-66:]
y2_test = daily_avg_temp['target_vent'].iloc[-66:]

model2 =LinearRegression()

model2.fit(X2_train, y2_train)

# Faire des prédictions sur l'ensemble de test
predictions2 = model2.predict(X2_test)

# Calculer les métriques de régression
mse2 = mean_squared_error(y2_test, predictions2)
rmse2 = mse2 ** 0.5  # Root Mean Squared Error
mae2 = mean_absolute_error(y2_test, predictions2)
r22 = r2_score(y2_test, predictions2)

# Afficher les métriques de régression
print('Mean Squared Error:', mse2)
print('Root Mean Squared Error:', rmse2)
print('Mean Absolute Error:', mae2)
print('R² Score:', r22)


Mean Squared Error: 47.14195258015155
Root Mean Squared Error: 6.865999750957726
Mean Absolute Error: 5.514109453174387
R² Score: 0.13731753702516758


In [13]:
# Afficher les prédictions et les valeurs réelles
predictions2 = pd.DataFrame({'Prédictions': predictions2, 'vent speed': y2_test})
predictions2 = predictions2.astype(int)
print(predictions2)


     Prédictions  vent speed
300           14          17
301           15          35
302           20          28
303           18          11
304           15          13
..           ...         ...
361           22          18
362           17          15
363           17          22
364           18          13
365           16          13

[66 rows x 2 columns]


## predict 10 days

In [14]:
import pandas as pd
import numpy as np

# Supposons que new_data est votre dataframe d'origine
new_data = pd.DataFrame({
    'Date': ['2024-05-27'],  # Date de départ spécifiée
    'Temp_C': [7],
    'Rel Hum_%': [86],
    'Wind Speed_km/h': [13]
})

new_data['Date'] = pd.to_datetime(new_data['Date'])

# Nombre de jours à prédire
days_to_predict = 10

# Créer un dataframe pour stocker toutes les prédictions
all_predictionss = pd.DataFrame(columns=['Date', 'Temp_C', 'Rel Hum_%', 'Wind Speed_km/h'])
all_predictionss = pd.concat([all_predictionss, new_data], ignore_index=True)

# Copier les données d'origine pour le processus de prédiction
data = new_data.copy()

for day in range(1, days_to_predict + 1):  
    # Prédire les valeurs pour le prochain jour
    prediction_vent = model2.predict(data.drop(['Date'], axis=1))
    prediction_hum = model1.predict(data.drop(['Date'], axis=1))
    prediction_tmp = model.predict(data.drop(['Date'], axis=1))
    
    temp_formatted = [format(temp, '.1f') for temp in prediction_tmp.flatten()]

    # Stocker les prédictions dans un dataframe temporaire
    temp_df = pd.DataFrame({
        'Date': [pd.to_datetime(data['Date'].iloc[0]) + pd.DateOffset(days=1)],  # Incrémenter la date
        'Temp_C': temp_formatted,
        'Rel Hum_%': prediction_hum.flatten().astype(int),
        'Wind Speed_km/h': prediction_vent.flatten().astype(int)
    })
    
    # Ajouter la nouvelle ligne à all_predictions
    all_predictionss = pd.concat([all_predictionss, temp_df], ignore_index=True)

    # Mettre à jour les données d'entrée pour la prochaine prédiction
    data = pd.concat([data, temp_df], ignore_index=True)
    data = data.iloc[1:]
all_predictionss['Date'] = all_predictionss['Date'].dt.strftime('%Y-%m-%d')

# Afficher toutes les prédictions
print(all_predictionss)


C:\Users\User\AppData\Local\Temp\ipykernel_1088\941825316.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_predictionss = pd.concat([all_predictionss, new_data], ignore_index=True)


          Date Temp_C Rel Hum_% Wind Speed_km/h
0   2024-05-27      7        86              13
1   2024-05-28    7.1        77              15
2   2024-05-29    7.0        71              15
3   2024-05-30    7.1        68              15
4   2024-05-31    7.3        67              15
5   2024-06-01    7.5        66              15
6   2024-06-02    7.7        65              14
7   2024-06-03    8.2        65              14
8   2024-06-04    8.6        65              14
9   2024-06-05    9.0        65              14
10  2024-06-06    9.4        65              14


In [15]:
from pymongo import MongoClient
import pandas as pd

# Connexion à MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["station_meteorologique"]
collection = db["predict 10 days"]

# Itérer sur le DataFrame et insérer chaque ligne dans la collection MongoDB
for index, row in all_predictionss.iterrows():
    prediction_doc = {
        "Date/Time": row["Date"],
        "Temp_C": row["Temp_C"],
        "Rel Hum_%": row["Rel Hum_%"],
        "Wind Speed_km/h": row["Wind Speed_km/h"],
    }

    # Insérer le document de prédiction dans la collection MongoDB
    insert_result = collection.insert_one(prediction_doc)
    
    # Vérifier le résultat de l'insertion
    if insert_result.acknowledged:
       print(f"Prédiction pour Date {row['Date']} insérée avec succès dans MongoDB.")
    else:
       print(f"Erreur lors de l'insertion de la prédiction pour Date {row['Date']} dans MongoDB.")
       print("Détails de l'erreur:", insert_result.raw_result)


Prédiction pour Date 2024-05-27 insérée avec succès dans MongoDB.
Prédiction pour Date 2024-05-28 insérée avec succès dans MongoDB.
Prédiction pour Date 2024-05-29 insérée avec succès dans MongoDB.
Prédiction pour Date 2024-05-30 insérée avec succès dans MongoDB.
Prédiction pour Date 2024-05-31 insérée avec succès dans MongoDB.
Prédiction pour Date 2024-06-01 insérée avec succès dans MongoDB.
Prédiction pour Date 2024-06-02 insérée avec succès dans MongoDB.
Prédiction pour Date 2024-06-03 insérée avec succès dans MongoDB.
Prédiction pour Date 2024-06-04 insérée avec succès dans MongoDB.
Prédiction pour Date 2024-06-05 insérée avec succès dans MongoDB.
Prédiction pour Date 2024-06-06 insérée avec succès dans MongoDB.
